In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd

### RMSD predicted-experimental

In [ ]:
pdb_af = pd.read_csv("PDB.exp-pred.alphafold.benchmark", sep="\t")
pdb_cf = pd.read_csv("PDB.exp-pred.colabfold.benchmark", sep="\t")
pdb = pd.concat([pdb_af.assign(algorithm="AlphaFold"), pdb_cf.assign(algorithm="ColabFold")])
pdb["Non-pruned"]=pdb["Pruned number of atom pairs"]/pdb["Total number of atom pairs"]*100
pdb

In [ ]:
os.chdir('runs')

In [ ]:
sns.set(style='whitegrid', rc={"xtick.bottom" : True, "ytick.left" : False})

f = sns.PairGrid(pdb.sort_values("PDB name", ascending=True),
                 x_vars=['RMSD (pruned)', 'RMSD (all)','Non-pruned'], y_vars=["PDB name"],
                 height=2.5, aspect=1.25, hue='algorithm')
f.map(sns.stripplot, size=10, orient="h" , jitter=False)

labels=['RMSD (pruned) [$\AA$]', 'RMSD (all) [$\AA$]', 'Post-pruning fraction [%]']
index=0
for ax, label in zip(f.axes.flat, labels):
    if index != 2:
        ax.axvline(2, ls="--", color="red")
    ax.xaxis.grid(True)
    ax.yaxis.grid(True)
    ax.set_xlabel(label)
    ax.xaxis.label.set_size(16)
    ax.yaxis.label.set_size(16)
    ax.minorticks_on()
    index+=1
plt.legend(loc='lower center', bbox_to_anchor=(-0.65, -0.9), fontsize=16)
sns.despine(left=True, bottom=True)
f.savefig("RMSD_exp-pred.svg")

### Dotplot AlphaFold vs. ColabFold

In [ ]:
data_0 = pd.read_csv("benchmark.comparison", sep="\t")
names = data_0["Protein"].tolist()
for i in range(len(names)):
    if len(names[i])==6:
        names[i]=names[i][:4]
    elif len(names[i])==4:
        continue
    else:
        split = names[i].split("_")
        species = '_'.join(split[:-1])
        loc = split[-1]
        loc = loc[:int(len(loc)/2)] + '-' + loc[int(len(loc)/2):]
        names[i] = species + ' [' + loc + ']'
data_0["Protein"]=names

In [ ]:
data_af = data_0[['Protein','pLDDT_AF','pTM_AF','Runtime_AF']]
data_af.columns = ['Protein', 'pLDDT', 'pTM', 'Runtime']
data_cf = data_0[['Protein','pLDDT_CF','pTM_CF','Runtime_CF']]
data_cf.columns = ['Protein', 'pLDDT', 'pTM', 'Runtime']
data = pd.concat([data_af.assign(algorithm='AlphaFold'), data_cf.assign(algorithm='ColabFold')])
data

In [ ]:
sns.set(style='whitegrid', rc={"xtick.bottom" : True, "ytick.left" : False})

g = sns.PairGrid(data.sort_values("Protein", ascending=True),
                 x_vars=data.columns[1:-1], y_vars=["Protein"],
                 height=7, aspect=.35, hue='algorithm')
g.map(sns.stripplot, size=10, orient="h" , jitter=False)

labels=['pLDDT [%]', 'pTM [-]', 'Runtime [s]']
axes_limits=[[60,101], [0.3,1.01], [0,22000]]
index=0
for ax, label in zip(g.axes.flat, labels):
    if index==0:
        ax.axvline(70, ls="--", color="red")
    elif index==1:
        ax.axvline(0.7, ls="--", color="red")
    ax.xaxis.grid(True)
    ax.yaxis.grid(True)
    ax.set_xlim(axes_limits[index][0], axes_limits[index][1])
    ax.set_xlabel(label)
    ax.xaxis.label.set_size(16)
    ax.yaxis.label.set_size(16)
    index+=1
plt.legend(loc='lower center', bbox_to_anchor=(-0.65, -0.25), fontsize=16)
sns.despine(left=True, bottom=True)
g.savefig("Dotplot-preds.svg")

### Pair-wise AlphaFold-ColabFold RMSD

In [ ]:
data_0['Non-pruned'] = data_0['Pruned number of atom pairs']/data_0['Total number of atom pairs']*100
data_0

In [ ]:
sns.set(style='whitegrid', rc={"xtick.bottom" : True, "ytick.left" : False})

h = sns.PairGrid(data_0.sort_values("Protein", ascending=True),
                 x_vars=['RMSD (pruned)', 'RMSD (all)', 'Non-pruned'], y_vars=["Protein"],
                 height=7, aspect=0.75)
h.map(sns.stripplot, size=10, orient="h" , jitter=False, color='blue')

labels = ['RMSD (pruned) [$\AA$]', 'RMSD (all) [$\AA$]', 'Post-pruning fraction [%]']
index=0
for ax, label in zip(h.axes.flat, labels):
    if index != 2:
        ax.axvline(2, ls="--", color="red")
    ax.xaxis.grid(True)
    ax.yaxis.grid(True)
    ax.set_xlabel(label)
    ax.xaxis.label.set_size(16)
    ax.yaxis.label.set_size(16)
    ax.minorticks_on()
    index+=1
sns.despine(left=True, bottom=True)
h.savefig("RMSD-preds.svg")